# Dynamische Arrays

Wir haben gesehen, wie wir die Methoden ```append``` und ```pop```, die ein Element am Ende eines Arrays anfügen, respektive entfernen, effizient implementieren können. Hier nochmals die Implementation:

In [ ]:
class  Array:
    
    def __init__(self):
        self.data = [None] # list  simulates  block  of  memory
        self.idx = 0
        
    def append(self, elem):
        if self.idx == len(self.data):    
            self.resize(len(self.data) * 2)
        self.data[self.idx] = elem
        self.idx  += 1        
    
    def pop(self):
        self.idx -= 1
        item = self.data[self.idx]; 
        if self.idx > 0  and self.idx == len(self.data) / 4:
            self.resize(int(len(self.data) / 2)) 

        return item; 

    def resize(self, numElements):
        newArray = [None] * numElements

        for i in  range(0, self.idx):            
            newArray[i] = self.data[i]
        self.data = newArray
    
    def __str__(self):
        return str(self.data)
    
    def length(self):
        return self.idx


Die folgenden Tests zeigen, dass die Implementation wie erwartet funktioniert:

In [ ]:
a = Array()
for i in range(0, 10):
    a.append(i)
print("Array after append: " + str(a))
print("length: " +str(a.length()))

print("idx before pop " +str(a.idx))
lastElement = a.pop()
print("lastElement " + str(lastElement))
print("Array after pop: " +str(a))
print("idx after pop " +str(a.idx))
print("length after pop: " +str(a.length()))

#### Experimente

Wir wollen nun testen, wie sich die Laufzeit der append Methode verhält. 

In [ ]:
import timeit
import random

In [ ]:
def createByAppend(n):
    a = Array()
    for i in range(0, n):
        a.append(i)
    return a;

In [ ]:
for n in [10, 100, 10000]:
    t = timeit.timeit(lambda: createByAppend(n), number=1)
    print("Durchschnittliche Zeit für eine append Operation bei " + str(n) + " Elementen = " + str(t / n))

Wie wir sehen, ist die Durchschnittliche Zeit pro ```append```-Operation konstant, unabhängig davon, wieviele Elemente wir einfügen. 

#### Miniübung 

* Vergleichen Sie die Laufzeit mit einer naiven Implementation, bei der das Array bei jedem Aufruf vergrössert wird.

### Ammortisierte Analyse: Array resizing

Im Folgenden schauen wir uns die ```append``` Operation noch auf theoretischer Ebene an. Der folgende Code dient zum illustrieren der Idee der amortisierten Analyse. Die Idee ist einfach - wir führen Buchhalten. Bei jeder ```billigen``` Operation erhalten wir einen Betrag, mit dem wir später die teure Operation bezahlen. Unsere Währung ist dabei ein *Token*. Für jeden Arrayzugriff in der resize Methode bezahlen wir mit einem Token. Wenn wir es schaffen, dass wir bei jeder ```append```-Operation immer dieselbe konstante Anzahl Tokens auf die Seite legen können, und damit genügend Tokens zur Verfügung haben um damit die Anzahl Array-Zugriffe in der ```resize``` Operation bezahlen können, haben wir eine amortisiert konstante Laufzeit.

Im folgenden Code ist diese Idee umgesetzt:

In [ ]:
class  ArrayWithAccounting:

    PRICE_IN_TOKENS_PER_APPEND = 1
    
    def __init__(self):
        self.data = [None] 
        self.idx = 0
        self.tokens = 0 # Anzahl Tokens ist am Anfang 0
        
    def append(self, elem):
        
        if self.idx == len(self.data):            
            newCapacity = len(self.data) * 2
            self.resize(newCapacity)
            print("number of tokens after resize to size " +str(newCapacity) +" = "+str(self.tokens))
        
        # Wir bezahlen eine Anzahl Tokens für jede Append Operation
        self.tokens += self.PRICE_IN_TOKENS_PER_APPEND
        
        self.data[self.idx] = elem
        
        print("number of tokens after append of element " +str(self.idx) + " = " +str(self.tokens))
              
        self.idx  += 1        

        

    def resize(self, numElements ):
        # Ein neues Array anzulegen braucht 1 Array Zugriffe pro Element
        # Wir bezahlen also ein Token pro Element
        newArray = [None] * numElements
        self.tokens -= 1 * numElements 
        
        for i in  range(0, self.idx):            
            newArray[i] = self.data[i]
            
            # Für jedes kopierte Element brauchen wir 2 Array Zugriffe 
            # Wir bezahlen also 2 Token pro Element
            self.tokens -= 2 
            
        self.data = newArray
    
    def __str__(self):
        return str(self.data)


Wenn wir diesen Code Ausführen, dann sehen wir, dass wir nach jeder Append Operation immer eine positive Anzahl Tokens haben. 

In [ ]:
a = ArrayWithAccounting()
for i in range(0, 32):
    a.append(i)


#### Miniübung: 

Wie viele Token brauchen Sie, damit Sie immer für die resize Operation bezahlen können?

#### Satz: Dynamisches vergrössern, verkleinern eines Arrays

Was wir nun experimentell herausgefunden haben lässt sich formal beweisen. Für eine Beweisskizze, siehe Sedgewick und Wayne, Algorithms, Satz E, Seite 221).

> Satz: Bei einem Array mit variabler Grösse ist gemäss obigen Algorithmus die durchschnittliche Anzahl der Arrayzugriffe für jede beliebige Folge von $M$ ```append``` und ```pop``` Operationen, ausgehend von dem leeren Array, proportional zu $M$.


### Dynamische Arrays in Python


Wie bereits im vorigen Notebook besprochen, ist ein Array in Python durch den Datentyp ```List``` implementiert. 
Der Datenbyp ```List``` entspricht einem dynamischen Array und unterstützt bereits die ```append``` und ```pop``` Methoden. Auch hier wollen wir wieder experimentell überprüfen, ob die Laufzeit unseren Erwartungen entspricht. 



In [ ]:
def createByAppendPython(n):
    a = []
    for i in range(0, n):
        a.append(i)
    return a;

In [ ]:
for i in range(1, 6):
    n = 10**i
 
    t = timeit.timeit(lambda: createByAppendPython(n), number=100)
    print("Durchschnittliche Zeit für eine append Operation bei " + str(n) + " Elementen = " + str(t / n))

Python stellt auch eine Methode zur Verfügung, um an einer beliebigen Stelle im Array ein Element einzufügen. Es ist leicht zu sehen, dass wir beim Einfügen an einer beliebigen Stelle, alle Elemente hinter der Einfügestelle kopieren müssen. Diese Operation hat also im schlimmsten Fall lineare Laufzeit in der Arraygrösse. Auch dies können wir einfach experimentell bestätigen

In [ ]:
def insertFirstPython(n):
    a = []
    for i in range(0, n):
        a.insert(0, i)
    return a;

In [ ]:
for i in range(1, 6):
    n = 10**i
    t = timeit.timeit(lambda: insertFirstPython(n), number=1)
    print("Durchschnittliche Zeit für eine insert Operation bei " + str(n) + " Elementen = " + str(t / n))